# GETTING STARTED

In [137]:
import sqlite3 as sql

In [138]:
conn = sql.connect("nominations.db")

In [139]:
cursor = conn.cursor()

# EXPLORING DATA

In [140]:
schema = cursor.execute("pragma table_info(\"nominations\");").fetchall()
for each in schema:
    print(each)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)


In [141]:
first_ten = cursor.execute("SELECT * FROM nominations LIMIT 10").fetchall()
for each in first_ten:
    print(each)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


# CREATING CEREMONIES TABLE

In [142]:
q = '''
    CREATE TABLE ceremonies (
        id integer PRIMARY KEY,
        Year integer,
        Host text
    )
'''

cursor.execute(q)

cursor.execute("SELECT * FROM ceremonies").fetchall()

[]

In [143]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]
q = "INSERT INTO ceremonies (Year, Host) VALUES (?,?)"
cursor.executemany(q,years_hosts)

cursor.execute("SELECT * FROM ceremonies").fetchall()

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin'),
 (11, 2000, 'Billy Crystal')]

In [144]:
cursor.execute("pragma table_info(ceremonies)").fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'Year', 'integer', 0, None, 0),
 (2, 'Host', 'text', 0, None, 0)]

# CHANGING NOMINATIONS CEREMONY REFERENCE
This is required for it to refer to the new ceremonies table

In [145]:
cursor.execute("pragma foreign_keys = ON")

In [146]:
q = "SELECT nominations.Category, nominations.Nominee, nominations.Movie, nominations.Character, nominations.Won, ceremonies.id\
    FROM nominations INNER JOIN ceremonies ON nominations.year == ceremonies.year"

joined_nominations = cursor.execute(q).fetchall()

for each in joined_nominations[:10]:
    print(each)

('Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1)
('Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1)
('Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1)
('Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1)
('Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1)
('Actor -- Supporting Role', 'Christian Bale', 'The Fighter', 'Dicky Eklund', 1, 1)
('Actor -- Supporting Role', 'John Hawkes', "Winter's Bone", 'Teardrop', 0, 1)
('Actor -- Supporting Role', 'Jeremy Renner', 'The Town', 'James Coughlin', 0, 1)
('Actor -- Supporting Role', 'Mark Ruffalo', 'The Kids Are All Right', 'Paul', 0, 1)
('Actor -- Supporting Role', 'Geoffrey Rush', "The King's Speech", 'Lionel Logue', 0, 1)


In [147]:
q = '''
    CREATE TABLE nominations_two (
        id integer PRIMARY KEY,
        category text,
        nominee text,
        movie text,
        character text,
        won integer,
        ceremony_id integer,
        FOREIGN KEY (ceremony_id) REFERENCES ceremonies(id)
    )
'''    

cursor.execute(q)

In [148]:
print(cursor.execute("SELECT * FROM nominations_two LIMIT 10").fetchall())

[]


In [149]:
q = '''
    INSERT INTO nominations_two
    (category, nominee, movie, character, won, ceremony_id)
     VALUES (?, ?, ? ,? ,? ,?)
'''
cursor.executemany(q,joined_nominations)

In [150]:
first_ten = cursor.execute("SELECT * FROM nominations_two LIMIT 10").fetchall()

for each in first_ten:
    print(each)

(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1)
(2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1)
(3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1)
(4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1)
(5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1)
(6, 'Actor -- Supporting Role', 'Christian Bale', 'The Fighter', 'Dicky Eklund', 1, 1)
(7, 'Actor -- Supporting Role', 'John Hawkes', "Winter's Bone", 'Teardrop', 0, 1)
(8, 'Actor -- Supporting Role', 'Jeremy Renner', 'The Town', 'James Coughlin', 0, 1)
(9, 'Actor -- Supporting Role', 'Mark Ruffalo', 'The Kids Are All Right', 'Paul', 0, 1)
(10, 'Actor -- Supporting Role', 'Geoffrey Rush', "The King's Speech", 'Lionel Logue', 0, 1)


In [151]:
conn.execute("DROP TABLE nominations")
conn.execute("ALTER TABLE nominations_two RENAME TO nominations")

first_ten = cursor.execute("SELECT * FROM nominations LIMIT 10").fetchall()

for each in first_ten:
    print(each)

(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1)
(2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1)
(3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1)
(4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1)
(5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1)
(6, 'Actor -- Supporting Role', 'Christian Bale', 'The Fighter', 'Dicky Eklund', 1, 1)
(7, 'Actor -- Supporting Role', 'John Hawkes', "Winter's Bone", 'Teardrop', 0, 1)
(8, 'Actor -- Supporting Role', 'Jeremy Renner', 'The Town', 'James Coughlin', 0, 1)
(9, 'Actor -- Supporting Role', 'Mark Ruffalo', 'The Kids Are All Right', 'Paul', 0, 1)
(10, 'Actor -- Supporting Role', 'Geoffrey Rush', "The King's Speech", 'Lionel Logue', 0, 1)


# NORMALIZING TABLES (MOVIES AND ACTORS)

In [152]:
q = '''
    SELECT * FROM sqlite_master WHERE type='table'
'''
cursor.execute(q).fetchall()

[('table',
  'ceremonies',
  'ceremonies',
  19,
  'CREATE TABLE ceremonies (\n        id integer PRIMARY KEY,\n        Year integer,\n        Host text\n    )'),
 ('table',
  'nominations',
  'nominations',
  20,
  'CREATE TABLE "nominations" (\n        id integer PRIMARY KEY,\n        category text,\n        nominee text,\n        movie text,\n        character text,\n        won integer,\n        ceremony_id integer,\n        FOREIGN KEY (ceremony_id) REFERENCES ceremonies(id)\n    )')]

In [153]:
q = '''
    CREATE TABLE movies (
    id integer PRIMARY KEY,
    movie text
    )
'''
cursor.execute(q)

q = '''
    CREATE TABLE actors (
    id integer PRIMARY KEY,
    actor text
    )
'''
cursor.execute(q)

q = '''
    CREATE TABLE movies_actors (
    id integer PRIMARY KEY,
    movie_id integer,
    actor_id integer,
    FOREIGN KEY (movie_id) REFERENCES movies(id),
    FOREIGN KEY (actor_id) REFERENCES actors(id)
    )
'''
cursor.execute(q)



In [154]:
q = '''
    SELECT * FROM sqlite_master WHERE type='table'
'''
cursor.execute(q).fetchall()

[('table',
  'ceremonies',
  'ceremonies',
  19,
  'CREATE TABLE ceremonies (\n        id integer PRIMARY KEY,\n        Year integer,\n        Host text\n    )'),
 ('table',
  'nominations',
  'nominations',
  20,
  'CREATE TABLE "nominations" (\n        id integer PRIMARY KEY,\n        category text,\n        nominee text,\n        movie text,\n        character text,\n        won integer,\n        ceremony_id integer,\n        FOREIGN KEY (ceremony_id) REFERENCES ceremonies(id)\n    )'),
 ('table',
  'movies',
  'movies',
  2,
  'CREATE TABLE movies (\n    id integer PRIMARY KEY,\n    movie text\n    )'),
 ('table',
  'actors',
  'actors',
  4,
  'CREATE TABLE actors (\n    id integer PRIMARY KEY,\n    actor text\n    )'),
 ('table',
  'movies_actors',
  'movies_actors',
  5,
  'CREATE TABLE movies_actors (\n    id integer PRIMARY KEY,\n    movie_id integer,\n    actor_id integer,\n    FOREIGN KEY (movie_id) REFERENCES movies(id),\n    FOREIGN KEY (actor_id) REFERENCES actors(id)\n  

In [155]:
q = '''
    SELECT DISTINCT(nominee) FROM nominations ORDER BY nominee ASC
'''
actors = cursor.execute(q).fetchall()

q = '''
    INSERT INTO actors (actor) VALUES (?)
'''
cursor.executemany(q,actors)

In [156]:
cursor.execute("SELECT * FROM actors LIMIT 5").fetchall()

[(1, 'Abigail Breslin'),
 (2, 'Adriana Barraza'),
 (3, 'Adrien Brody'),
 (4, 'Alan Alda'),
 (5, 'Alan Arkin')]

In [157]:
q = '''
    SELECT DISTINCT(movie) FROM nominations ORDER BY movie ASC
'''
movies = cursor.execute(q).fetchall()

q = '''
    INSERT INTO movies (movie) VALUES (?)
'''
cursor.executemany(q,movies)

In [158]:
cursor.execute("SELECT * FROM movies LIMIT 5").fetchall()

[(1, '127 Hours'),
 (2, '21 Grams'),
 (3, 'A Beautiful Mind'),
 (4, 'A History of Violence'),
 (5, 'A Single Man')]

In [159]:
q = '''
    SELECT movies.id, actors.id FROM nominations 
    INNER JOIN actors ON nominations.nominee==actors.actor
    INNER JOIN movies ON nominations.movie==movies.movie
'''
movies_actors = cursor.execute(q).fetchall()

movies_actors[:10]

[(16, 66),
 (130, 67),
 (121, 72),
 (108, 31),
 (1, 64),
 (104, 25),
 (139, 76),
 (122, 71),
 (107, 96),
 (108, 48)]

In [162]:
q = '''
    INSERT INTO movies_actors (movie_id,actor_id) VALUES (?,?)
'''
cursor.executemany(q,movies_actors)

In [164]:
q = '''
    SELECT movies.movie,actors.actor FROM movies
    INNER JOIN movies_actors ON movies.id = movies_actors.movie_id
    INNER JOIN actors ON movies_actors.actor_id = actors.id
    LIMIT 10
'''
cursor.execute(q).fetchall()

[('Biutiful', 'Javier Bardem'),
 ('True Grit', 'Jeff Bridges'),
 ('The Social Network', 'Jesse Eisenberg'),
 ("The King's Speech", 'Colin Firth'),
 ('127 Hours', 'James Franco'),
 ('The Fighter', 'Christian Bale'),
 ("Winter's Bone", 'John Hawkes'),
 ('The Town', 'Jeremy Renner'),
 ('The Kids Are All Right', 'Mark Ruffalo'),
 ("The King's Speech", 'Geoffrey Rush')]

In [165]:
conn.close()